# PL ANALYSIS
* Hi! This notebook is for analysis of PL data. 
* I am assuming your data is organised in this format: [Path to all the data]\\[name of sample]\\[Pixel number]. 
    * These will have a bunch of images, a folder called 'refs', a folder called 'white'and a csv file called IV
    
### Some instructions: 
* Please run each cell in order, following the instructions
* Have at hand your datapath, and the path you would like to save the data in
    * The outputs will sometimes show here but all of it is saved in the folder you provide
* Press cntrl+enter to run a cell
* When prompted enter information into the feilds and press enter
* If you go back and re run a cell, that's fine but you have to run every cell after that again, and in order

This might break if all these files are not there! If you want to edit this code definately go ahead, a bunch of the functions live in seperate .py files, BUT please make a copy!!!

SEE LAST PANEL FOR BASIC HELP IF YOU HAVE DIFFICULTY

In [ ]:
# Some imports:
from external_imports import *
from image_process import *
from general_data_process import *
from reconstruct_jv import *

# For parallel processing
import multiprocessing
num_cores = multiprocessing.cpu_count()

# Plot parameters
import matplotlib
matplotlib.rcParams['figure.dpi'] = 100
matplotlib.rcParams['font.size'] = 12
matplotlib.rcParams['font.family'] = "Century Gothic"

### Data + Save folders
* Copy and paste the path to your data bellow, as well as where you would like the output files from this analysis to be saved
* Don't remove the " " marks, or the 'r' in front of them, that's important!
* Again, make sure your data is organised in the "[Path to all the data]\[name of sample]\[Pixel number]" format

In [ ]:
datapath = r"C:\Users\akashdasgupta\OneDrive - Nexus365\Data\EL_setup\actual_but_reduced"
savepath = r"C:\Users\akashdasgupta\OneDrive - Nexus365\Data\EL_setup\actual_processed"

path_db = path_process(datapath)

### STEP 3: Enter bandgaps
* We need bandgaps for the calculations
* Run bellow, enter as it prompts 

In [ ]:
bandgaps = {}
for key in path_db.keys():
    if key.lower() != 'white':
        string = "What's the bandgap of "+key+" in eV?: "
        bandgap = float(input(string))
        bandgaps[key] = bandgap

### STEP 5: Calculate corrections

In [ ]:
with open(f"{datapath}\\white_params.csv", 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        if row[0].lower() == "exposure":
            white_exposure = float(row[1])
        elif row[0].lower() == "flux scale":
            white_flux = float(row[1])
        
correction_db = {}
for key in path_db.keys(): 
    bandgap = bandgaps[key]
    correction = white_over_cell_correction(white_exposure, ledspecf, np.vectorize(BBf_cellf), 
                                            bandgap, camqef, lenscalf, 0.99, filtcalf)

    correction_db[key] = correction

## Different analysis modules:
* The following will be different cells that do different bits of analysis, just run the ones you want

### STEP 6.1 (opt.): Whole image averaged JV
* Calculate Psudo J/V and Sun's Voc JV averaging over whole cropped image

In [ ]:
for key in path_db.keys():
    for pix in path_db[key]:     

        path = f"{datapath}\\{key}\\{pix}"
        savepath_i = f"{savepath}\\{key}\\{pix}"

        bandgap = bandgaps[key]

        white_mean_scaled = np.mean(np.load(f"{datapath}\\{key}\\{pix}\\white.npy"))/(correction_db[key]*white_flux)
        flux_1sun = j1sunf(bandgap)
        voc_rad = vocradf(bandgap)

        num_suns, rrs, Vs, Js, fluxes = whole_image_jv(path, flux_1sun, voc_rad, white_mean_scaled)

        
        ##########
        # Suns voc plot
        suns_voc_V = []
        with open(f"{datapath}\\{key}\\{pix}\\oc\\source_meter.csv",'r') as file:
            reader = csv.reader(file)
            for row in reader:
                suns_voc_V.append(float(row[0]))
        suns_voc_V = np.array(suns_voc_V)
        num_suns_suns_voc = deepcopy(num_suns)
        num_suns_suns_voc.sort()
        num_suns_suns_voc = np.flip(num_suns_suns_voc)
        
        #####
        # Plot
        
        plt.scatter(-Vs, Js, color='k', label = 'Reconstructed')
        plt.plot(suns_voc_V, 1-num_suns_suns_voc, color='r', label = "Sun's Voc measurement")
        
        plt.xlim((-1.3,-0.3))      
        plt.ylim((-1,1))
        plt.axhline(0,color='k')
        
        plt.legend(frameon=False)
        plt.xlabel("Voltage (V)")
        plt.ylabel("J/J$_{sc}$")
        plt.title(f"{key}, Pixel {pix}")
        
        if not os.path.isdir(f"{savepath_i}\\whole_im_reconstructed_jvs"):
            os.makedirs(f"{savepath_i}\\whole_im_reconstructed_jvs")
        plt.savefig(f"{savepath_i}\\whole_im_reconstructed_jvs\\PL_reconstructed.png")
        
        plt.show()
        
        ###############
        # saves as csv
        

        with open(f"{savepath_i}\\whole_im_reconstructed_jvs\\PL_reconstructed.csv", 'w', newline='') as file:
            writer = csv.writer(file)
            for row in zip(-Vs, Js):
                writer.writerow(row)
        with open(f"{savepath_i}\\whole_im_reconstructed_jvs\\suns_voc.csv", 'w', newline='') as file:
            writer = csv.writer(file)
            for row in zip(suns_voc_V, 1-num_suns_suns_voc):
                writer.writerow(row)


### STEP 6.2(opt.): Whole image spatially resolved JV
* Calculate Psudo J/V per pixel, making a set of images of some current, with a spatial voltage map 
* Option to interpolate this, and make images at constant voltage, mapping current at that voltage per pixel

In [ ]:
for key in path_db.keys():
    for pix in path_db[key]:     

        path = f"{datapath}\\{key}\\{pix}"
        savepath_i = f"{savepath}\\{key}\\{pix}"

        bandgap = bandgaps[key]

        white_mean_scaled = np.mean(np.load(f"{datapath}\\{key}\\{pix}\\white.npy"))/(correction_db[key]*white_flux)
        flux_1sun = j1sunf(bandgap)
        voc_rad = vocradf(bandgap)

        array_jv(path, savepath_i, flux_1sun, voc_rad, white_mean_scaled)

In [ ]:
# Run this to save the images
for key in path_db.keys():
    for pix in path_db[key]:
        path = f"{savepath}\\{key}\\{pix}\\V"
        files = find_npy(path)

        for file in files:
            savefile_name = '_'.join(file.split('_')[:-1])+'.png'
            fig, ax = plt.subplots()
            J_led = 1 - float(file.split('_')[1])

            imarr = np.load(f"{path}\\{file}")
            im = ax.imshow(imarr, cmap="inferno")
            ax.set_title('J/J$_{sc}$ ='+f' {J_led}')
            cbar = plt.colorbar(im,pad=0.01,aspect=20)
            cbar.set_label('V$_{oc}$ (V)', rotation=270,labelpad=25)

            scalebar_size = 0.1 # cm
            scalebar_offset = 10 # pix

            scalebar_width_pix = 0.1*pixels_per_cm
            scalebar_height_pix = scalebar_width_pix/10
            scalbar_y_position = imarr.shape[0]-scalebar_offset-scalebar_height_pix

            text_x = scalebar_offset + scalebar_width_pix/2
            text_y = scalbar_y_position

            scalebar = patches.Rectangle((scalebar_offset,scalbar_y_position),scalebar_width_pix,scalebar_height_pix,linewidth=1,edgecolor='k',facecolor='white')
            ax.add_patch(scalebar)
            txt = plt.text(text_x,text_y,"1 mm", color = "white", ha='center', va='bottom', fontweight='bold')
            txt.set_path_effects([PathEffects.withStroke(linewidth=1.2, foreground='k')])

            plt.axis('off')

            plt.savefig(f"{path}\\{savefile_name}")
            
            plt.close('all')

#### STEP 6.2.1(opt.): Interpolate JV from before
* You need to have ren the cell above for this to work
* Here, for each pixel we reconstruct a JV curve from before, and construct a bunch of images at constant voltage along the JV curve, mapping spatial current
* THIS TAKES A LONG TIME TO RUN!!! Took 1/2 hour per pixel on 9th gen i7, 8 cores (utalising parallel processing)

In [ ]:
for key in path_db.keys():
    for pix in path_db[key]:     
        path = f"{savepath}\\{key}\\{pix}"
        jv_extrapolator(path, num_cores)

In [ ]:
# Run this to save the images
for key in path_db.keys():
    for pix in path_db[key]:
        path = f"{savepath}\\{key}\\{pix}\\V_inter"
        files = find_npy(path)

        for file in files:
            fig, ax = plt.subplots()
            v = '.'.join(file.split('.')[:-1])

            imarr = np.load(f"{path}\\{file}")
            im = ax.imshow(imarr, cmap="inferno")
            ax.set_title(f'V={v}')
            cbar = plt.colorbar(im,pad=0.01,aspect=20)
            cbar.set_label('J/<J${sc}$>', rotation=270,labelpad=25)

            scalebar_size = 0.1 # cm
            scalebar_offset = 10 # pix

            scalebar_width_pix = 0.1*pixels_per_cm
            scalebar_height_pix = scalebar_width_pix/10
            scalbar_y_position = imarr.shape[0]-scalebar_offset-scalebar_height_pix

            text_x = scalebar_offset + scalebar_width_pix/2
            text_y = scalbar_y_position

            scalebar = patches.Rectangle((scalebar_offset,scalbar_y_position),scalebar_width_pix,scalebar_height_pix,linewidth=1,edgecolor='k',facecolor='white')
            ax.add_patch(scalebar)
            txt = plt.text(text_x,text_y,"1 mm", color = "white", ha='center', va='bottom', fontweight='bold')
            txt.set_path_effects([PathEffects.withStroke(linewidth=1.2, foreground='k')])

            plt.axis('off')

            plt.savefig(f"{path}\\{v}.png")
            plt.close('all')